In [14]:
import tensorflow as tf
import pickle
import numpy as np
import time
from load_cifar import *

In [15]:
valid_features, valid_labels = load_preprocessed_validation_batch()

test_features, test_labels = load_preprocessed_test_batch()

In [16]:
tf.reset_default_graph()

<h1>Hyperparameter</h1>

In [17]:
#learning rate
lr = 0.0001
#number of traning steps
num_epochs =10

#number of batch_size
batch_size = 128
#network parameters
num_classes = 10

<h1>Placeholder</h1>

In [18]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='X')
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='Y')
drop_prob = tf.placeholder(tf.float32, name='drop_prob')

<h1>LeNet-5</h1>

In [19]:
def LeNet5(X,drop_prob):
    conv1_W = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], mean=0, stddev=0.08))
    conv1_b = tf.Variable(tf.zeros(shape=64))

    conv1 = tf.nn.conv2d(X, conv1_W, strides=[1,1,1,1], padding='SAME')+ conv1_b
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)
    
    conv2_W = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], mean=0, stddev=0.08))
    conv2_b = tf.Variable(tf.constant(0.1,shape=[64]))

    conv2 = tf.nn.conv2d(conv1_bn, conv2_W, strides=[1,1,1,1], padding='SAME')+ conv2_b
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    conv3_W = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 128], mean=0, stddev=0.08))
    conv3_b = tf.Variable(tf.constant(0.1,shape=[128]))
        
    conv3 = tf.nn.conv2d(conv2_bn, conv3_W, strides=[1,1,1,1], padding='SAME') +conv3_b
    conv3 = tf.nn.relu(conv3)
    conv3_pool = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=[5, 5, 128, 256], mean=0, stddev=0.08))
    conv3_b = tf.Variable(tf.constant(0.1,shape=[256]))
    
    conv4 = tf.nn.conv2d(conv3_pool, conv4_W, strides=[1,1,1,1], padding='SAME') +conv3_b
    conv4 = tf.nn.relu(conv4)
    conv4_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    flat = tf.contrib.layers.flatten(conv4_pool)  

    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, drop_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, drop_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=512, activation_fn=tf.nn.relu)
    full3 = tf.nn.dropout(full3, drop_prob)
    full3 = tf.layers.batch_normalization(full3)    
    
    full4 = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=120, activation_fn=tf.nn.relu)
    full4 = tf.nn.dropout(full4, drop_prob)
    full4 = tf.layers.batch_normalization(full4) 
    
    logits = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=10, activation_fn=tf.nn.softmax)
    return logits

<h1>Cost and Optimization</h1>

In [20]:
logits = LeNet5(X,drop_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
train_optimizer = optimizer.minimize(loss)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.global_variables_initializer()


<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [21]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        #fetch batch
        start = time.time()
        for batch_id in np.arange(1,6):
            for fearues_batch, labels_batch in load_preprocessed_training_batch(batch_id,batch_size):
                #run optimization
                sess.run(train_optimizer, feed_dict={X:fearues_batch, 
                                                     Y:labels_batch,
                                                     drop_prob: 0.5})
        end = time.time()
        x_train,y_train = pickle.load(open('cifar-preprocessed/batch_train_1','rb'))
        acc = sess.run(accuracy,feed_dict={X:valid_features, 
                                           Y:valid_labels,
                                           drop_prob: 1.})
        print("epoch "+str(i)+", Validation Accuracy= {:.3f}".format(acc))
        print("    Train Time = {:.3f}".format(end-start))
    print("Training finished!") 
    print("Testing ACcuracy:", sess.run(accuracy, 
                                        feed_dict={X:test_features, 
                                                   Y:test_labels, 
                                                   drop_prob: 1.}))

epoch 0, Validation Accuracy= 0.298
    Train Time = 228.783
epoch 1, Validation Accuracy= 0.380
    Train Time = 245.853
epoch 2, Validation Accuracy= 0.423
    Train Time = 229.710
epoch 3, Validation Accuracy= 0.445
    Train Time = 247.904
epoch 4, Validation Accuracy= 0.470
    Train Time = 227.784
epoch 5, Validation Accuracy= 0.487
    Train Time = 246.126
epoch 6, Validation Accuracy= 0.496
    Train Time = 260.556
epoch 7, Validation Accuracy= 0.507
    Train Time = 256.174
epoch 8, Validation Accuracy= 0.526
    Train Time = 254.951
epoch 9, Validation Accuracy= 0.537
    Train Time = 219.242
Training finished!
Testing ACcuracy: 0.5421
